In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd 'drive/My Drive/Colab Notebooks/Mnist-Model-Deploy'

/content/drive/My Drive/Colab Notebooks/Mnist-Model-Deploy


In [3]:
!pip install flask-ngrok

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from flask import Flask, flash, redirect, render_template, request, url_for, send_from_directory,abort
from flask_ngrok import run_with_ngrok
from tensorflow.keras.models import load_model

mnist_model = load_model('model/mnist.h5')

In [0]:
app = Flask(__name__)
run_with_ngrok(app) 
app.secret_key = 'vatsalparsaniya'

app.config["MNIST_BAR"] = "generated_image/mnist_vis"
app.config["IMAGES"] = "upload"

@app.route('/')
def home():
    flash("Hello Welcome to Vatsal's ML-DL Model Deploy site")
    return render_template('index.html')

@app.route('/mnist/')
def mnist_home():
    return render_template('mnist.html')

@app.route('/mnistprediction/', methods=['GET', 'POST'])
def mnist_prediction():
    if request.method == "POST":
        if not request.files['file'].filename:
            flash("No File Found")
        else:
            f =  request.files['file']
            f.save("uploads/"+f.filename)
            image_gray  = cv2.imread("uploads/"+f.filename, cv2.IMREAD_GRAYSCALE)
            img_resize = cv2.resize(image_gray,(28,28))
            image_bw = cv2.threshold(img_resize, 75, 255, cv2.THRESH_BINARY)[1]
            bitwise_not_image = cv2.bitwise_not(image_bw, mask=None)
            pred_img = np.reshape(bitwise_not_image,(1,28,28,1))/255.0

            predictions = mnist_model.predict(pred_img)
            number = int(np.argmax(predictions))
            print(number)

            plt.figure()
            y_pos = np.arange(10)
            plt.bar(y_pos, predictions[0])
            plt.savefig('generated_image/mnist_vis/'+f.filename)

            return str(number)

@app.route("/get-mnist-image/<image_name>")
def get_mnist_image(image_name):
    try:
        return send_from_directory(app.config["MNIST_BAR"], filename=image_name)
    except FileNotFoundError:
        abort(404)

In [6]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://23a9d0a9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/May/2020 15:08:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:01] "GET /static/css/header_file.css HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:01] "GET /static/css/footer_file.css HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:01] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:01] "GET /static/image/mnist-sample.png HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/May/2020 15:08:07] "GET /mnist/ HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:07] "GET /static/css/mnist_home.css HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:07] "GET /static/js/imagebox_mnist.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:07] "GET /static/js/predictbtn_mnist.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2020 15:08:08] "GET /mnist/ HTTP/1.1" 200 -
